In [ ]:
import numpy
import time
import os


file_dir_path = "./output/"

file_name_list = os.listdir(file_dir_path)
data_list = []
# 载入所有npy文件
for file_name in file_name_list:
    data_list.append(numpy.load(file_dir_path+file_name,allow_pickle=True).item())

In [ ]:
import h3


# 按照小区分析
resolution = 5
h3_dict = {}
for i in range(len(data_list)):
    # 对每一个文件i，即旧数据
    for sat in data_list[i].keys():
        # 对每一个卫星
        for j in range(len(data_list[i][sat]['pos_bl'])):
            # 对每一次记录
            pos_bl = data_list[i][sat]['pos_bl'][j]
            h3_index = h3.geo_to_h3(pos_bl[0] / numpy.pi * 180, pos_bl[1] / numpy.pi * 180, resolution)
            record = [
                data_list[i][sat]['epoch'][j],
                data_list[i][sat]['pos'][j],
                data_list[i][sat]['velocity'][j],
                data_list[i][sat]['altitude'][j],
                data_list[i][sat]['radial velocity'][j],
                data_list[i][sat]['pos_bl'][j]
            ]
            if h3_index in h3_dict.keys():
                h3_dict[h3_index].append(record)
            else:
                h3_dict[h3_index] = [record]

In [12]:
# 将按照小区归类后的数据存储
numpy.save("analyse_output/ByRegion_analyse.npy", h3_dict)

In [ ]:
print(len(h3_dict.keys()))
# print(h3_dict['8597086bfffffff'])
# 统计每个小区有多少条卫星记录
h3_count = {}
for h3_index in h3_dict.keys():
    h3_count[h3_index] = len(h3_dict[h3_index])
# 按照卫星数量从大到小的顺序对小区进行排序
h3_count_sorted = sorted(h3_count.items(), key=lambda x:x[1], reverse=True)

In [ ]:
# 把最多记录的小区的卫星记录输出csv，用于查看
import pandas


df = pandas.DataFrame(h3_dict[h3_count_sorted[0][0]])
df.columns = [
    'epoch',
    'pos',
    'velocity',
    'altitude',
    'radial velocity',
    'pos_bl',
]
df.to_csv(h3_count_sorted[0][0]+"_data.csv")

In [14]:
# 统计每个小区有多少条卫星记录输出成csv
df_count = pandas.DataFrame(h3_count_sorted)
df_count.columns = [
    'h3_index',
    'count'
]
df_count.to_csv("count_data.csv")